# **Data Wrangling**
En este notebook realizo las tareas identificadas en el **EDA**, entre ellas:
- Asignación de columnas que me interesan del INDEC (`Variedad`, `Unidad de medida` y `Octubre de 2025`)
- Limpieza de valores nulos
- Formateo a `snake_case` y eliminación de tildes
- Pasado de USD a pesos

**KPIs:**
- Desperdicio general -> % y $
- Ganancia de categorías afectada -> % (ej: pérdida de ganancia de un 7% en [categoría])
- Comparación temporal de ventas con y sin optimización

**Insights:**
- Productos con más pérdidas 

## 1. Lectura de archivos _staging_ e importación de librerias
- Definición de la carpeta raíz e importación de librerias

In [11]:

import sys, os

# Obtengo la ruta raíz del proyecto (subo un nivel desde /notebooks)
project_root = os.path.abspath("../")

# La agrego al path si no está
if project_root not in sys.path:
    sys.path.append(project_root)

# Importado de módulos y librerías
from scripts.utils import pasar_snake_case, aplicar_funcion_df, transformar_unidades, traductor_cols
import pandas as pd
import matplotlib.pyplot as plt

- Lectura de archivos

In [12]:
dir_staging = r'..\data\staging'

# Lectura de CSVs 
df_orders = pd.read_csv(dir_staging + r'\orders.csv')
df_items = pd.read_csv(dir_staging + r'\items_traducido.csv')
df_indec_temporal = pd.read_csv(dir_staging + r'\indec.csv')

- Asignación de columnas

In [ ]:
df_indec = pd.DataFrame()

# Asignación de columnas
df_indec['variedad'] = df_indec_temporal.iloc[:, 0]
df_indec['unidad_de_medida'] = df_indec_temporal.iloc[:, 1]
df_indec['precio'] = df_indec_temporal.iloc[:, 2]

# Revisión rápida
print(df_indec.isnull().sum().sum()) 

              Unnamed: 0 Unnamed: 1 Unnamed: 2     Precios  Unnamed: 3  \
0                    NaN  Unidad de        NaN         NaN         NaN   
1               Variedad        NaN        NaN         NaN         NaN   
2                    NaN     medida    Octubre  Septiembre         NaN   
3                    NaN        NaN    de 2025     de 2025         NaN   
4                    NaN        NaN        NaN       Pesos         NaN   
..                   ...        ...        ...         ...         ...   
59               Algodón      100 g   1.648,05    1.635,68         NaN   
60                Champú     400 cc   5.870,70    5.779,51         NaN   
61           Desodorante     150 cc   3.534,84    3.414,64         NaN   
62      Jabón de tocador      125 g   1.151,60    1.132,33         NaN   
63  Pañales descartables       10 u   4.389,07    4.223,13         NaN   

          Variación  
0        porcentual  
1               NaN  
2   respecto al mes  
3          anterior  
4

## 2. Limpieza de datos

- Eliminación de valores nulos

In [14]:
df_items = df_items.dropna() # aunque ya son eliminados en "traducir_df.py"
df_indec = df_indec.dropna() 

## 3. Conversiones de datos

- Traducción de columnas del inglés al español

In [15]:
# Traduzco las columnas
for df in [df_orders, df_items]:
    df.columns = [traductor_cols(col) for col in df.columns.tolist()]

> **Nota:** La traducción del DataFrame general es ejecutada en un script aparte (`traducir_df.py`). Para más información, acceder a él.

- Pasado a `snake_case` con una función auxiliar

In [16]:
# Convertir valores a snake_case solo para ciertas columnas
aplicar_funcion_df(df_items, ["categoria", "subcategoria", "nombre_producto"], pasar_snake_case)
aplicar_funcion_df(df_indec, ["variedad", "unidad_de_medida", "precio"], pasar_snake_case) # la columna precio es una string, reemplazo "," y "."

- Conversión de precios y unidades de medida a kilogramos

In [17]:
# Conversión a float
df_indec["precio"] = df_indec["precio"].astype(float)

# Utilizo función auxiliar que se encarga de pasar precios de diversas unidades/medidas a kg
df_indec = transformar_unidades(df_indec)

c:\Users\User\Desktop\analisis-de-desperdicios\scripts\utils.py:103: UserWarning: Valores no reconocidos en transformar_unidades: [('hamburguesas_congeladas', 'env_4_u', 'Unidad desconocida'), ('salchicha_tipo_viena', 'env_6_u', 'Unidad desconocida'), ('huevos_de_gallina', 'docena', 'Formato desconocido'), ('polvo_para_flan', '8_porciones', 'Unidad desconocida'), ('panales_descartables', '10_u', 'Unidad desconocida')]
  warnings.warn(f"Valores no reconocidos en transformar_unidades: {avisos}")


- Conversión de tipo de cambio (dólares a pesos)

In [ ]:

# Obtengo valor del dólar
with open(dir_staging + r"\dolar_blue.txt", "r") as db:
    valor_dolar = float(db.readline())

# Aplico cálculos
aplicar_funcion_df(df_items, ["precio", "costo_produccion"], lambda cant: round((cant * valor_dolar), 1))


## 4. Paso a mercado argentino
Probablemente lo más desafiante del proyecto

**Manera de afrontarlo**

La manera en la que decidí afrontar la generalidad de este 

                                   variedad unidad_de_medida        precio
0                   pan_frances_tipo_flauta               kg   3908.680000
1                               pan_de_mesa               kg   8577.076923
2   galletitas_dulces_envasadas_sin_relleno               kg   8578.266667
3              galletitas_de_agua_envasadas               kg   4931.160000
4                 harina_de_trigo_comun_000               kg    914.610000
5                       arroz_blanco_simple               kg   1702.330000
6                 fideos_secos_tipo_guisero               kg   3032.440000
7                                     asado               kg  11775.550000
8                        carne_picada_comun               kg   7320.710000
9                                    paleta               kg  12018.190000
10                                  cuadril               kg  14686.830000
11                                    nalga               kg  15191.590000
12                       